# Julia Language Features

## Introduction and Background

* Created by Jeff Bezanson, Alan Edelman, Stefan Karpinski, and Viral B. Shah in 2009
* [Released publicly in 2012](https://julialang.org/blog/2012/02/why-we-created-julia)
* [Version 1.0 released August 2018](https://julialang.org/blog/2018/08/one-point-zero)
* Current version is 1.3.1 released December 30, 2019

#### Main language features

* Multiple dispatch (parametric polymorphism)
* Dynamic type system ("optional" typing)
* High performance (approaching C, Fortran, etc.)
* Built-in package manager
* Lisp-like macros and metaprogramming
* Interoperability with Python, R, C, Fortran
* Designed for parallel and distributed computing

In this notebook we're going to just hit a few highlights and just scratch the surface.

## Julia is *Fast*

### Python vs Julia

#### Riddler Express from FiveThirtyEight

https://fivethirtyeight.com/features/so-you-want-to-tether-your-goat-now-what/

> From Luke Robinson, a serenading stumper:

> My daughter really likes to hear me sing “The Unbirthday Song” from “Alice in Wonderland” to her. She also likes to sing it to other people. Obviously, the odds of my being able to sing it to her on any random day are 364 in 365, because I cannot sing it on her birthday. The question is, though, how many random people would she expect to be able to sing it to on any given day before it became more likely than not that she would encounter someone whose birthday it is? In other words, what is the expected length of her singing streak?

*Aside:* The problem is asking two different questions. The minimum number of random people that would make it more likely than not the girl would encounter someone whose birthday it is is the **median**, while the expected length of the singing streak is the **mean** of the distribution of singing streak lengths. We'll just calculate the **mean**.

First let's look at a **Python** simulation to calculate the approximate expected length of the singing streak.

```python
from random import randrange
from statistics import mean
import time

def trial():
    n = 0
    singing = True
    while (singing):
        if (randrange(365) == 0):
            singing = False
        else:
            n += 1
    return n

def do_trials(n_trials):
    trials = [None] * n_trials
    for i in range(n_trials):
        trials[i] = trial()
    return mean(trials)

n_trials = int(1e6)
print(f"Performing {n_trials:,} trials...")

start_time = time.time()
print(do_trials(n_trials))
end_time = time.time()
print("Elapsed time was %g seconds" % (end_time - start_time))
```

Output:

```
% python unbirthday.py           
Performing 1,000,000 trials...
363.791765
Elapsed time was 185.602 seconds
```

That's a little over 3 minutes.

Then let's compare a **Julia** simulation.

In [ ]:
using Statistics

function trial()
    n = 0
    singing = true
    while (singing)
        if (rand(1:365) == 1)
            singing = false
        else
            n += 1
        end
    end
    return n
end

function do_trials(n_trials)
    trials = zeros(Int, n_trials)
    for i in 1:n_trials
        trials[i] = trial()
    end
    mean(trials)
end

In [ ]:
@time begin
    n_trials = Int(1e6)
    result = do_trials(n_trials)
    println("Expected number of days: $result")
end

That's fast!

-------------

#### A better (exact) solution

The [Geometric distrubution](https://en.wikipedia.org/wiki/Geometric_distribution) is the probability distribution of the number $Y$ of failures of Bernoulli trials before the first success. The probability mass function for the Geometric distribution is

$${\Pr(Y=k)=(1-p)^{k}p}$$

for k = 0, 1, 2, 3, .... where $p$ is probability of success for each Bernoulli trial.

The mean of the Geometric distribution is

$$E(Y) = \frac{1 - p}{p}$$

In our case, $p$ is the probability that a random person we encounter has a birthday today, so

$$p = \frac{1}{365}$$

and therefore

$$E(Y) = \frac{1 - \frac{1}{365}}{\frac{1}{365}}$$

$$ = 365 - 1$$

$$ = 364$$

## Why does Julia sometimes feel slow?

But we've already seen that Julia can *feel* slow when used interactively. Packages can be slow to load, and functions can be slow the first time they are called. What's happening is that Julia code is compiled "just in time" and that compilation can take a little time. Julia's maintainers are well aware of the issue of "compilation latency" and plan to address it in the future.

In the meantime, everyone will have to decide for themselves if the tradeoffs that Julia makes are worth it for a particular application.


# Selected Julia language features

There is a lot more than we can cover today.

## Variables

In [ ]:
x = 1
y = 2
x + y

### Unicode and LaTeX variable names

In [ ]:
# Korean
안녕하세요 = "Hello"

In [ ]:
α = 1.0
β₁ = 2.0
β₂ = 3.0

α + β₁ * 5.0 + β₂ * 3.5

In [ ]:
π

### Even emojis

Please don't ever do this. 😉

In [ ]:
😺 = "Smiley cat"

In [ ]:
typeof(😺)

In [ ]:
😺 = 1

In [ ]:
typeof(😺)

In [ ]:
😀 = 0
😞 = -1

In [ ]:
😺 + 😞 == 😀

### Aside

In [ ]:
0.1 + 0.2 == 0.3

In [ ]:
0.1 + 0.2 ≈ 0.3

# Selected data structures

1. Tuples
2. Dictionaries
3. Arrays


### Tuples

In [ ]:
my_favorite_languages = ("Julia", "Python", "R")

In [ ]:
my_favorite_languages[1]

Tuples are immutable, so it's an error to try this:

In [ ]:
my_favorite_languages[3] = "Ruby"

### Dictionaries

In [ ]:
d1 = Dict(1 => 4.2, 2 => 5.3)

In [ ]:
keys(d1)

In [ ]:
values(d1)

In [ ]:
d1[2]

In [ ]:
d2 = Dict(1 => 4.2, :two => "hello") 

In [ ]:
d2["whatever"] = true
d2

We can be explict about types.

In [ ]:
d3 = Dict{Symbol, Int64}(:a => 1, :b => 2, :c => 3)

Being explicit about types helps prevent certain kinds of bugs and also generally improves performance.

This will be an error because the types don't match.

In [ ]:
d3["whatever"] = true

### Arrays

In [ ]:
fibonacci = [1, 1, 2, 3, 5, 8, 13]

In [ ]:
mixture = [1, 1, 2, 3, "Ted", "Robyn"]

In [ ]:
push!(fibonacci, 21)

In [ ]:
fibonacci

In [ ]:
pop!(fibonacci)

In [ ]:
fibonacci

Assigment is by reference, so be careful.

In [ ]:
somenumbers = fibonacci
somenumbers[3] = 999

In [ ]:
fibonacci

To avoid this, use the ```copy``` function.

Multiple dimensional arrays are also supported.

In [ ]:
rand(4, 3)

In [ ]:
rand(4, 2, 3)

# Control Flow

* Loops
* Array comprehensions

## Loops

### while loop

In [ ]:
n = 0
while n < 5
    n += 1
    println(n)
end
n

### for loop

In [ ]:
for n in 1:5
    println(n)
end

Let's create an addition table using some syntactic sugar for a nested for loop.

In [ ]:
m, n = 5, 5
A = fill(0, (m, n))

In [ ]:
for i in 1:m, j in 1:n
    A[i, j] = i + j
end
A

## Array comprehensions

Here is the same thing in more idiomatic Julia using an *array comprehension*.

In [ ]:
B = [i + j for i in 1:m, j in 1:n]

# Functions

This section is from Jane Herriman's [Introduction to Julia Tutorials](https://github.com/xorJane/Introduction_to_Julia_tutorials)

Topics:
1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

## How to declare a function
Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords

In [ ]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

In [ ]:
function f(x)
    x^2
end

We can call either of these functions like this:

In [ ]:
sayhi("C-3PO")

In [ ]:
f(42)

Alternatively, we could have declared either of these functions in a single line

In [ ]:
sayhi2(name) = println("Hi $name, it's great to see you!")

In [ ]:
f2(x) = x^2

In [ ]:
sayhi2("R2D2")

In [ ]:
f2(42)

Finally, we could have declared these as "anonymous" functions

In [ ]:
sayhi3 = name -> println("Hi $name, it's great to see you!")

In [ ]:
f3 = x -> x^2

In [ ]:
sayhi3("Chewbacca")

In [ ]:
f3(42)

In [ ]:
(x -> x^3)(3)

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."* <br><br>
Julia functions will just work on whatever inputs make sense. <br><br>
For example, `sayhi` works on the name of this minor tv character, written as an integer...

In [ ]:
sayhi(55595472)

And `f` will work on a square matrix. 

In [ ]:
A = rand(3, 3)
A

In [ ]:
f(A)

`f` will also work on a string like "hi" because `*` is defined for string inputs as string concatenation.

In [ ]:
f("hi")

On the other hand, `f` will not work on a vector. Unlike `A^2`, which is well-defined, the meaning of `v^2` for a vector, `v`, is not a well-defined algebraic operation. 

In [ ]:
v = rand(3)

In [ ]:
f(v)

## Mutating vs. non-mutating functions

By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.


In [ ]:
v = [3, 5, 2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged. <br><br>

On the other hand, when we run `sort!(v)`, the contents of v are sorted within the array `v`.

In [ ]:
sort!(v)

In [ ]:
v

## Some higher order functions

### map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguments. 
`map` then applies that function to every element of the data structure you pass it. For example, executing

```julia
map(f, [1, 2, 3])
```
will give you an output array where the function `f` has been applied to all elements of `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [ ]:
map(f, [1, 2, 3])

Here we've squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as

In [ ]:
x -> x^3

via

In [ ]:
map(x -> x^3, [1, 2, 3])

and now we've cubed all the elements of `[1, 2, 3]`!

### broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do every thing `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`

In [ ]:
broadcast(f, [1, 2, 3])

and again, we've applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`!

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

```julia
broadcast(f, [1, 2, 3])
```
is the same as
```julia
f.([1, 2, 3])
```

In [ ]:
f.([1, 2, 3])

Notice again how different this is from calling 
```julia
f([1, 2, 3])
```
We can square every element of a vector, but we can't square a vector!

To drive home the point, let's look at the difference between

```julia
f(A)
```
and
```julia
f.(A)
```
for a matrix `A`:

In [ ]:
A = [i + 3*j for j in 0:2, i in 1:3]

In [ ]:
f(A)

As before we see that for a matrix, `A`,
```
f(A) = A^2 = A * A
``` 

On the other hand,

In [ ]:
B = f.(A)

contains the squares of all the entries of `A`.

This dot syntax for broadcasting allows us to write relatively complex compound elementwise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

In [ ]:
A .+ 2 .* f.(A) ./ A

instead of

In [ ]:
broadcast(x -> x + 2 * f(x) / x, A)

and the two will perform exactly the same.

# Type system and multiple dispatch

This section is also from Jane Herriman's [Introduction to Julia Tutorials](https://github.com/xorJane/Introduction_to_Julia_tutorials)

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [ ]:
f(x) = x^2

and then Julia will determine on its own which input argument types make sense and which do not:

In [ ]:
f(10)

In [ ]:
f([1, 2, 3])

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [ ]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [ ]:
foo("hello", "hi!")

In [ ]:
foo(3, 4)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [ ]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

In [ ]:
foo(3, 4)

Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [ ]:
foo("hello", "hi!")

This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)```

Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [ ]:
methods(foo)

Aside: how many methods do you think there are for addition?

In [ ]:
methods(+)

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [ ]:
@which foo(3, 4)

Let's see what happens when we use `@which` with the addition operator!

In [ ]:
@which 3.0 + 3.0

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [ ]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

This method for `foo` will work on, for example, floating point numbers:

In [ ]:
foo(3.0, 4.0)

We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [ ]:
foo(x, y) = println("I accept inputs of any type!")

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [ ]:
v = rand(3)
foo(v, v)

# Metaprogramming

From Wikipedia:

> In computer programming, **homoiconicity** (from the Greek words homo- meaning "the same" and icon meaning "representation") is a property of some programming languages. A language is **homoiconic** if a program written in it can be manipulated as data using the language, and thus the program's internal representation can be inferred just by reading the program itself. For example, a Lisp program is written as a regular Lisp list, and can be manipulated by other Lisp code.[1] In homoiconic languages, all code can be accessed and transformed as data, using the same representation. This property is often summarized by saying that the language treats "code as data".

Julia is homoiconic. In Julia, program code can be represented by a Julia data structure called an expression.

In [ ]:
Meta.parse("2 + 3")

In [ ]:
typeof(ans)

In [ ]:
two_plus_three = :(2 + 3)

This is known as *quoting* and `:` is the `quote` operator.

Code with more than one line can be quoted like this:

In [ ]:
quote
    a = 42
    b = a^2
    a - b
end

In [ ]:
eval(two_plus_three)

In [ ]:
dump(two_plus_three)

`head` indicates that this expression is a function call. `args` is an array containing the function and its arguments.

In [ ]:
two_plus_three.args[1]

Let's make a copy of the expression and modify the first `arg`

In [ ]:
two_minus_three = copy(two_plus_three)
two_minus_three.args[1] = :-

In [ ]:
two_plus_three

Now we can look at and evaluate the modified expression.

In [ ]:
two_minus_three

In [ ]:
eval(two_minus_three)

## Macro example

Using the above and just a little more metaprogramming machinery, we can create powerful *macros*.

Here is an example.

In [ ]:
macro timeit(ex)
    quote
        local t0 = time()
        local val = $(esc(ex))
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

In [ ]:
@timeit factorial(20)

Note: Julia comes with a built-in `@time` macro.

In [ ]:
@time factorial(20)

# Taylor series example

Based on an example by Mike J Innes.

## Original sin

Here's a more practical example. Consider the following definition of the `sin` function, based on the Taylor series.

$$sin(x) = \sum_{k=1}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

In [ ]:
mysin(x) = sum((-1)^k/factorial(1+2k) * x^(1+2k) for k = 0:5)

In [ ]:
mysin(0.5), sin(0.5)

To see where we are right now, we'll benchmark it.

In [ ]:
using BenchmarkTools
@benchmark mysin(0.5)

Right now, this is much slower than it could be. The reason is that we're looping over `k`, which is relatively expensive. It'd be much faster to write out:

In [ ]:
mysin(x) = x - x^3/6 + x^5/120 # + ...

But this is tedious to write, and no longer looks like the original Taylor series. It's harder to tell if we've made a mistake, and we easily modify it. Is there a way to get the best of both worlds?

How about getting Julia to write out that code for us?

To start with, let's consider a symbolic version of the `+` function.

In [ ]:
plus(a, b) = :($a + $b)

In [ ]:
plus(1, 2)

With `plus` we can do more interesting things, like symbolic `sum`:

In [ ]:
reduce(+, 1:10)

In [ ]:
reduce(plus, 1:10)

In [ ]:
eval(ans)

Given that, we can also sum over symbolic variables.

In [ ]:
reduce(plus, [:(x^2), :x, 1])

This gives us an important piece of the puzzle, but we also need to figure out _what_ we're summing. Let's crate a symbolic version of the Taylor series above, which interpolates the value of `k`.

In [ ]:
k = 2
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

Now we have one term, we can generate as many as we like.

In [ ]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]

And sum them –

In [ ]:
reduce(plus, ans)

And create a function definition out of it:

In [ ]:
:(mysin(x) = $ans)

In [ ]:
eval(ans)

In [ ]:
mysin(0.5), sin(0.5)

In [ ]:
@benchmark mysin(0.5)

Compare this to the benchmark results above.